# Database Snapshots

dbatools makes using database snapshots easier than ever.

- Take a snapshot of database
- Drop a table or run a rogue update
- Get the table back

## View our database snapshots
Database snapshots have a `source_database_id` value that relates to the database they were created from.

In [ ]:
select name, database_id, source_database_id, is_read_only
from sys.databases
where name like 'AdventureWorks2017%'

## View the current state

In [ ]:
USE AdventureWorks2017

SELECT TOP 10 *
FROM Person.Password

SELECT TOP 10 *
FROM Person.PersonPhone


## Time for some fun - Rogue changes

In [ ]:
Use AdventureWorks2017

DROP TABLE person.PASSWORD
GO

UPDATE Person.PersonPhone
SET PhoneNumber = '330-333-3333'

## Uh-oh - my data!

In [ ]:
USE AdventureWorks2017

SELECT TOP 10 *
FROM Person.Password

SELECT TOP 10 *
FROM Person.PersonPhone


## Let's Fix this from our snapshot
We have two options here
1. Revert the whole databases to the point in time the snapshot was taken
1. Pull certain data out of the snapshot to fix our mistakes

### Option 1 - Get the data to fix PersonPhone

In [ ]:
-- Clear out the data
DELETE 
FROM AdventureWorks2017.Person.PersonPhone

-- Insert the data from the table within our snapshot
INSERT INTO Person.PersonPhone
SELECT *
FROM AdventureWorks2017_20211007_055506.Person.PersonPhone

-- Check the data
SELECT TOP 10 * 
FROM AdventureWorks2017.Person.PersonPhone


### Option 2 - Revert to the time the snapshot was taken
Back to PowerShell...